# 3-Qubit bit flip Quantum Error Correction Code

This notebook demonstrates the 3-Qubit bit flip Quantum Error Correction Code from Chapter 13, Quantum Error Correction: A Primer.

## 0. Setup

The cells below install the packages and import the libraries needed by this demo.

In [ ]:
#@title Install Qiskit and other dependencies
%pip install qiskit qiskit-aer qiskit-ibm-runtime pylatexenc

In [21]:
# Imports

import math
from ipywidgets import interact
from IPython.display import display, clear_output
from qiskit_aer import AerSimulator
from qiskit_ibm_runtime import SamplerV2 as Sampler
from qiskit.circuit import AncillaRegister, ClassicalRegister, QuantumCircuit, QuantumRegister
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

## 1. Circuit Definition

The following cell defines the 3-Qubit Bit Flip Quantum Error Correction Code circuit.

Refer to **2. Interaction** for configurable options (parameters).

In [22]:
def create_circuit(ry=0.0, unitary_op=None, error_bit=-1):
    q = [q1, q2, q3] = QuantumRegister(3, name='q')
    a = [a1, a2] = AncillaRegister(2, name='a')
    o = ClassicalRegister(3, name='output')
    s = [s1, s2] = ClassicalRegister(2, name='syndrome')
    circuit = QuantumCircuit(q, a, o, s)

    if ry != 0.0:
        circuit.ry(ry, q1)
    circuit.cx(q1, q2)
    circuit.cx(q1, q3)
    circuit.barrier(label='Ψ₁')
    if unitary_op == 'X':
        circuit.x(q1)
        circuit.x(q2)
        circuit.x(q3)
    circuit.barrier(label='Ψ₂')
    if error_bit != -1:
        circuit.x(q[error_bit])
    circuit.barrier(label='Ψ₃')
    circuit.cx(q1, a1)
    circuit.cx(q2, a1)
    circuit.cx(q1, a2)
    circuit.cx(q3, a2)
    circuit.measure(a, s)
    with circuit.if_test((s1, 1)):
        with circuit.if_test((s2, 1)):
            circuit.x(q1)
    with circuit.if_test((s1, 1)):
        with circuit.if_test((s2, 0)):
            circuit.x(q2)
    with circuit.if_test((s1, 0)):
        with circuit.if_test((s2, 1)):
            circuit.x(q3)

    circuit.barrier(label='Ψ₄')
    circuit.cx(q1, q3)
    circuit.cx(q1, q2)
    circuit.measure(q, o)
    return circuit

def run_circuit(circuit):
    backend = AerSimulator()
    pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
    isa_circuit = pm.run(circuit)
    sampler = Sampler(backend)
    job = sampler.run([isa_circuit], shots=1000)
    result = job.result()

    output_counts = result[0].data.output.get_counts()
    for bits, val in sorted(output_counts.items()):
        assert bits[0:2] == "00"
        bit = bits[2]
        print(f"{bit}\t{val}")

    syndrome_counts = result[0].data.syndrome.get_counts()
    assert len(syndrome_counts) == 1
    syndrome = next(iter(syndrome_counts.keys()))
    print(f"{syndrome=}")

## 2. Interaction

The cell below displays and runs variants of the circuit for the 3-Qubit Bit Flip Quantum Error Correction Code defined in **1. Circuit Definition**.

You can experiment with these parameters:

* `ry`: The RY angle for the initialization state. The default input uses an angle of 0, which provides the $|0\rangle$ state.
* `unitary_op`: Optionally apply a logical unitary operation $X_L$ to the input state. 
* `error_op`: Single qubit to apply a bit flip error to: -1 = no error, 0-2 = error on qubit 0-2.

In [23]:
def run_interaction(ry=0.0, unitary_op=None, error_bit=-1, diagram=True):
    clear_output(wait=True)
    circuit = create_circuit(ry, unitary_op, error_bit)
    if diagram:
        display(circuit.draw(output="mpl", fold=-1, cregbundle=True))
    else:
        run_circuit(circuit)

interact(run_interaction, ry=(-math.pi, math.pi, math.pi/8), unitary_op=[None, 'X'], error_bit=(-1, 2), diagram=True)

interactive(children=(FloatSlider(value=0.0, description='ry', max=3.141592653589793, min=-3.141592653589793, …

<function __main__.run_interaction(ry=0.0, unitary_op=None, error_bit=-1, diagram=True)>